<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">COMP3611 - Building a Machine Learning Pipeline</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Marc de Kamps and University of Leeds</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

# Building a Machine Learning Pipeline (Part 3)

## Objectives

Here, we finally analyse the data. We will experiment with different regressors, preform cross validation and parameter search and will use the *scikit-learn* interface to do so.

In particular, we will 
- Apply linear regression to make predictions
- Apply decision trees to make predictions
- Apply random forests to make predictions
- Compare these different regression methods ob the mean squared error criterion
- Perform cross validation and grid search

In [2]:
import os
import numpy as np
import pandas as pd
import tarfile

from sklearn.model_selection import StratifiedShuffleSplit

local_path = 'datasets/housing'


def restore():
    housing_tgz=tarfile.open(os.path.join(local_path,'./housing.tgz'))
    housing_tgz.extractall(path=local_path)
    housing_tgz.close()

    csv_path=os.path.join(local_path,'./housing.csv')
    housing = pd.read_csv(csv_path)

    # create test training set with stratified sampling (see previous notebook)
    housing["income_category"]=np.ceil(housing["median_income"]/1.5)
    housing["income_category"].where(housing["income_category"] < 5, 5.0, inplace = True)

# Stratified ShuffleSplit cross-validator
# Provides train/test indices to split data in train/test sets.
# This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. 
# The folds are made by preserving the percentage of samples for each class.
    
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=42)

    # returns 2 sets of indexes for test and train
    # hence, .loc is used on the dataset df to retrieve the corresponding records
    for train_index, test_index in split.split(housing,housing["income_category"]):
        strat_train_set = housing.loc[train_index]
        strat_test_set = housing.loc[test_index]
    
    for set_ in (strat_train_set, strat_test_set):
        set_.drop(("income_category"),axis=1,inplace=True)
        
   
    return strat_train_set, strat_test_set

strat_train_set, strat_test_set = restore()
strat_train_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,<1H OCEAN
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,INLAND
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,<1H OCEAN
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,<1H OCEAN


In [28]:
type(strat_train_set)

pandas.core.frame.DataFrame

In [3]:
housing=strat_train_set.drop("median_house_value",axis=1)
housing_labels=strat_train_set["median_house_value"].copy()

In [4]:
housing_num=housing.drop("ocean_proximity",axis=1)

In [5]:
num_attribs=list(housing_num) # list of variable names for numerical variables
cat_attribs=["ocean_proximity"] # list categorical variable names

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy="median")

In [8]:
from sklearn.base import BaseEstimator

class CombinedAttributesAdder(BaseEstimator):

    def __init__(self, do_add_bedrooms_per_room = False):
        
        # simply a binary variable per room
        self.do_add_bedrooms_per_room = do_add_bedrooms_per_room
        
        # These are the column indices of the respective columns. OK for illustration purposes.
        # For more robust code you would want to extract these values from the DataFrame by name.
        self.rooms_ix      = 3
        self.bedrooms_ix   = 4
        self.population_ix = 5
        self.household_ix  = 6
        
    def fit(self, X, y=None):
        # We don't transform the target values here
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:,self.rooms_ix]/X[:,self.household_ix]
        population_per_household = X[:, self.population_ix]/ X[:,self.rooms_ix]
        if self.do_add_bedrooms_per_room:
            bedrooms_per_room = X[:,self.bedrooms_ix]/X[:,self.rooms_ix]
            return np.c_[X,rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        

In [9]:
class DataFrameSelector(BaseEstimator):
    
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
        
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values



In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
num_pipeline= Pipeline([
    ('selector', DataFrameSelector(num_attribs)), # dataframe selector just gets the list of numerical variable names
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler()) # standard scaler minuses mean and divides by S.D -> results in a gaussian-like
    # distribution where mean is 0 and S.D. is 1
])

In [12]:
cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('one hot',OneHotEncoder())
])

In [13]:
from sklearn.pipeline import FeatureUnion
# feature union concatenates the results of each pipeline
# (numerical and categorical pipeline outputs)
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline",num_pipeline),
    ("cat_pipeline",cat_pipeline)
])

In [14]:
housing_prepared = full_pipeline.fit_transform(housing)

In [15]:
housing_prepared

<16512x15 sparse matrix of type '<class 'numpy.float64'>'
	with 181632 stored elements in Compressed Sparse Row format>

In [16]:
housing_prepared.shape

(16512, 15)

### Regression
Up until this point, the pipeline is a succcint version of all we did in Part 2, with one small exception that we encourage you to find. *housing_prepared* is fully imputed and processed version of the test set. *housing_labels* the associated labesl.  The code for linear regression is shockingly simple.

In [17]:
from sklearn.linear_model import LinearRegression

lin_reg=LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

It is always a good idea to see what this looks like on the dataset:

In [18]:
some_data=housing.iloc[:5]
some_labels=housing_labels.iloc[:5]
some_data_prepared=full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

Predictions: [ 88370.94144057 304135.24349272 154102.43801373 183742.54559955
 247301.43535955]
Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [19]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse=mean_squared_error(housing_labels, housing_predictions)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

68916.20007477468


It does something sensible, but the root mean squared error is sizeable. Although some of this is caused by the deviation of prediction and outcome for the more expensive homes, inspection of individual predictions shows substantial deviations for some data points. And this is on the **training set**.

Let's try a decsion tree prediction

In [20]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()

tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

Note the similarity of interface. Prediction also looks similar.

In [21]:
housing_predictions=tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rsme=np.sqrt(tree_mse)
tree_rsme

0.0

No error at all! Actually this is somewhat suspect. There is no doubt the tree has learnt the training set perfectly, but this may indicate overfitting.

### Cross validation
In cross validation the dataset is partitioned. In n-fold cross validation, the following experimnt is repeated n times: set a fraction of n-1/n patterns apart from training and use the rest for evaluation. This gives n different scores. THe implementation is straightforward with only one caveat: *scikit-learn* expects a utility function (lower is better than higher) rather than a cost function, and the scoring function is the negative of the MSE. The code below compensates for that.

In [22]:
from sklearn.model_selection import cross_val_score

scores =cross_val_score(tree_reg, housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)

tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print('Scores:',scores)
    print('Mean:',scores.mean())
    print('Standard deviation:',scores.std())
    
display_scores(tree_rmse_scores)

Scores: [68213.19692098 67617.18327208 66237.30469979 71836.70028509
 65102.33951135 75505.32294097 71615.25160317 74404.51768303
 69841.13568462 70193.07297589]
Mean: 70056.6025576962
Standard deviation: 3205.257773081889


If anything, the performance is now slightly worse than for linear regression. This is a clear example of overfitting by the original tree.

**Exercise** Carry out cross validation for the linear regression

Now, we will try a random forest regressors.

In [23]:
from sklearn.ensemble import RandomForestRegressor
forest_reg=RandomForestRegressor()
forest_reg.fit(housing_prepared,housing_labels)

housing_predictions=forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rsme=np.sqrt(forest_mse)
forest_rsme

KeyboardInterrupt: 

This looks encouraging: a better error than linear regression, but no clear overfitting. Again, let's cross validate.

In [ ]:
scores=cross_val_score(forest_reg, housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores=np.sqrt(-scores)

display_scores(forest_rmse_scores)

## Grid Search

This is clearly better than the linear regressor. We have used the random forest 'out of the box', not bothering to tweak its parameters. The lectures should have given you some ideas on how different parameter settings can be used to alter decision trees and random forests. In a *grid search* you systematically try out combinations of relevant parameters. As you can check from the *scikit-learn* documentation, you can vary (at least) the following parameters:

- n_estimators
- max_features
- bootstrap

Suppose you want to systematically explore a number of parameter settings, then *GridSearchCV* can help you do this:

In [24]:
from sklearn.model_selection import GridSearchCV

param_grid  = [{'n_estimators': [3, 10, 30],'max_features':[2, 4, 6, 8]}, 
               {'bootstrap': [False],'n_estimators':[3,10],'max_features':[2,3,4]}]

forest_reg=RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             scoring='neg_mean_squared_error')

After this has finished, you need to get the best scoring model:

In [25]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [26]:
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30)

The evaluation scores are available:

In [27]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

64289.59241974437 {'max_features': 2, 'n_estimators': 3}
56034.82086494566 {'max_features': 2, 'n_estimators': 10}
52248.268463309316 {'max_features': 2, 'n_estimators': 30}
60245.83897102302 {'max_features': 4, 'n_estimators': 3}
53152.88388023122 {'max_features': 4, 'n_estimators': 10}
49979.54008063302 {'max_features': 4, 'n_estimators': 30}
58348.14674002388 {'max_features': 6, 'n_estimators': 3}
51836.676088965694 {'max_features': 6, 'n_estimators': 10}
49207.81634318747 {'max_features': 6, 'n_estimators': 30}
57705.21990446179 {'max_features': 8, 'n_estimators': 3}
51080.414713257385 {'max_features': 8, 'n_estimators': 10}
49354.106548158285 {'max_features': 8, 'n_estimators': 30}
61543.99146641973 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54246.50558414164 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59573.71395872397 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52109.097964656336 {'bootstrap': False, 'max_features': 3, 'n_estimators

You should explore the possibility to do a **RandomizedSearchCV**.

**Exercise** Why is this sometimes a better option? When would you use it?

### Feature Importance

It can be useful to explore feature importance:

In [28]:
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs= ["rooms_per_hhold", "pop_per_hhold","bedrooms_per_room"]

# Here we use the OneHotEncoder class to retrieve the original categories
encoder=OneHotEncoder()
encoder.fit(housing[cat_attribs])
cat_one_hot_attribs = [ el for el in encoder.categories_[0]]
print(cat_one_hot_attribs)
attributes=num_attribs + extra_attribs + cat_one_hot_attribs


['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']


In [29]:
sorted(zip(feature_importances,attributes),reverse=True)

[(0.325585596532664, 'median_income'),
 (0.15409029094590615, '<1H OCEAN'),
 (0.14525985830517613, 'pop_per_hhold'),
 (0.08521176386580434, 'longitude'),
 (0.08287846473390835, 'latitude'),
 (0.069779631605732, 'rooms_per_hhold'),
 (0.040186516019098166, 'housing_median_age'),
 (0.019759932786445555, 'total_rooms'),
 (0.019589907041384073, 'population'),
 (0.019430641319843758, 'total_bedrooms'),
 (0.018310606447502074, 'households'),
 (0.012865090304563185, 'bedrooms_per_room'),
 (0.0036701775194709976, 'ISLAND'),
 (0.0033499285476077585, 'NEAR BAY'),
 (3.159402489358889e-05, 'INLAND')]

Interestingly, the feature that a house must be close to the ocean seems to be the only important categorical feature. You can consider leaving out the the others.


## Evaluation on the test set

This is a step you always should do. If the results are substantially worse than the cross validated ones, you still must distrust your model.

In [25]:
final_model=grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value",axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions=final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test,final_predictions)
final_rsme = np.sqrt(final_mse)
print(final_rsme)

In [27]:
y_test

5241     500001.0
17352    162500.0
3505     204600.0
7777     159700.0
14155    184000.0
           ...   
12182    110000.0
7275     118800.0
17223    293800.0
10786    335700.0
3965     215600.0
Name: median_house_value, Length: 4128, dtype: float64

This looks reasonably close to the cross validated results on the training set.